In [1]:
import numpy as np
import pandas as pd
from utility import *

homedir = get_homedir()
FIPS_mapping, FIPS_full = get_FIPS(reduced=True)

## Reading data

In [2]:
import json

with open(f'{homedir}/JK/po_code_state_map.json') as f:
    po_st = json.load(f)

st_to_fips = {}
for dic in po_st:
    st_to_fips[dic['state']] = dic['fips']
st_to_fips['California']    

'06'

In [3]:
demo = pd.read_csv(f'{homedir}/JK/preprocessing/0508/demographic.csv')
demo.loc[:, 'fips'] = demo['fips'].apply(correct_FIPS)
FIPS_demo = set(demo['fips'])

In [4]:
policy_date = ['County Emergency Declaration Date', 'Safer-at-Home Policy Date', 'Business Closure Policy Date']
policy1 = pd.read_csv(f'{homedir}/JK/County_Declaration_and_Policies.csv', usecols=['FIPS']+policy_date, parse_dates=policy_date)
policy1['FIPS'] = policy1['FIPS'].apply(correct_FIPS)
policy1.replace({'FIPS':FIPS_mapping})
policy1.head()

,FIPS,County Emergency Declaration Date,Safer-at-Home Policy Date,Business Closure Policy Date
0,01001,NaT,NaT,NaT
1,01003,2020-03-14,NaT,NaT
2,01005,NaT,NaT,NaT
3,01007,NaT,NaT,NaT
4,01009,2020-03-17,NaT,NaT


In [5]:
policy2 = pd.read_csv(f'{homedir}/data/us/other/policies.csv', usecols=['FIPS', 'stay at home'])
policy2['stay at home'] = policy2['stay at home'].apply(lambda x:(pd.Timestamp.fromordinal(int(x)) if not np.isnan(x) else x))
# policy2['stay at home'] = policy2['stay at home'].apply(pd.Timestamp.fromordinal)
policy2['FIPS'] = policy2['FIPS'].apply(correct_FIPS)
policy2.replace({'FIPS':FIPS_mapping})
policy2.head()

,FIPS,stay at home
0,0,NaT
1,01000,NaT
2,01001,NaT
3,01003,NaT
4,01005,NaT


In [6]:
policy3 = pd.read_csv('https://raw.githubusercontent.com/covidvis/covid19-vis/master/data/quarantine-activity-US-Apr16.csv', usecols=list(range(1,12)), parse_dates=[1])
col_pol3 = list(policy3.columns)[:4]+[list(policy3.columns)[5]]+list(policy3.columns)[9:]
policy3['State of Emergency Declaration'] = ~policy3['State of Emergency Declaration'].isna()
policy3.replace({'Shelter-in-place Order':{'Night-time curfew':np.nan}}, inplace=True)
policy3['Shelter-in-place Order'] = ~(policy3['Shelter-in-place Order'].isna())
policy3['business'] = ~((policy3['Bar and Dine-in Restaurant Closure'].isna()) & (policy3['Non-essential Businesses Closure'].isna()))
policy3 = policy3[col_pol3+['business']]
policy3.drop(columns=col_pol3[-2:], inplace=True)
policy3.replace({'State':st_to_fips}, inplace=True)
policy3.head()

,State,Effective Date,Coverage,State of Emergency Declaration,Shelter-in-place Order,business
0,41,2020-02-28,State-wide,False,False,False
1,53,2020-02-29,State-wide,True,False,False
2,12,2020-03-01,State-wide,True,False,False
3,01,2020-03-02,State-wide,False,False,False
4,06,2020-03-04,State-wide,True,False,False


In [7]:
coverage_df = policy3[policy3['Coverage']!='State-wide'][['State', 'Coverage']]
coverage_list = list(coverage_df['Coverage'])
coverage_list

['Scarsdale',
 'Osceola',
 'King, Snohomish, and Pierce',
 'Saline, Jefferson, Pulaski, and Grant',
 'Marin, Sacramento, San Joaquin, San Luis Obispo, Santa Clara, Solano, Placer, and Contra Costa, as well as the Oakland, Antioch, Santa Cruz, Los Angeles Unified, Chaffey Unified, Etiwanda, Fontana Unified, Ontario-Montclair, Alta Loma Unified, San Diego, Los Alamitos Unified, and Washington Unified School Distrcts.',
 'Wellesley, Framingham, and Boston',
 'Dutchess, Orange, Herkimer, Ulster, and Warwick',
 'St Paul',
 'Hoboken',
 'Davidson',
 'New York City, Westchester, Suffolk, and Nassau',
 'Burlington',
 'Tucson and Phoenix',
 'Alameda, Contra Costa, Marin, San Francisco, San Mateo, and Santa Clara.',
 'Blaine',
 'Jackson, Johnson, Kansas City',
 'New York City',
 'San Miguel',
 'Athens-Clarke',
 'Honolulu and Maui',
 'Cass, Douglas, and Sarpy',
 'Chattanooga',
 'Maricopa, Pinal, Navajo, Coconino, and Graham',
 'Kauai',
 'New Orleans',
 'Knoxville',
 'Columbus',
 'Everett',
 'Ancho

In [8]:
import re

cov_parsed = []
for cov in coverage_list:
    _ = re.sub(",* and", ",", re.sub(" as well as", "", cov))
    cov_parsed.append(_.strip().split(', '))

In [11]:
list_st = list(coverage_df['State'])
excep = []
ctylist = [[] for _ in range(len(list_st))]
for i in range(len(list_st)):
    dic_instate = fips_key[fips_key['ST']==list_st[i]][['FIPS', 'COUNTY']]
    for j in range(len(cov_parsed[i])):
        bo = True
        for idx, row in dic_instate.iterrows():
            if row['COUNTY'][:len(cov_parsed[i][j])]==cov_parsed[i][j]:
                print(cov_parsed[i][j], ':', row['FIPS'])
                bo = False
                ctylist[i].append(row['FIPS'])
                break
        if bo:
            print('Not found')
            excep.append([list_st[i], cov_parsed[i][j]])

Not found
Not found
King : 53033
Snohomish : 53061
Pierce : 53053
Saline : 05125
Jefferson : 05069
Pulaski : 05119
Grant : 05053
Marin : 06041
Sacramento : 06067
San Joaquin : 06077
San Luis Obispo : 06079
Santa Clara : 06085
Solano : 06095
Placer : 06061
Contra Costa : 06013
Not found
Not found
Santa Cruz : 06087
Not found
Not found
Not found
Not found
Not found
Not found
San Diego : 06073
Not found
Not found
Not found
Not found
Not found
Dutchess : 36027
Orange : 36071
Herkimer : 36043
Ulster : 36111
Not found
Not found
Not found
Davidson : 47037
Not found
Westchester : 36119
Suffolk : 36103
Nassau : 36059
Not found
Not found
Not found
Alameda : 06001
Contra Costa : 06013
Marin : 06041
San Francisco : 06075
San Mateo : 06081
Not found
Blaine : 16013
Jackson : 20085
Johnson : 20091
Not found
Not found
San Miguel : 08113
Not found
Honolulu : 15003
Maui : 15009
Cass : 31025
Douglas : 31055
Sarpy : 31153
Not found
Maricopa : 04013
Pinal : 04021
Navajo : 04017
Coconino : 04005
Graham : 04

In [13]:
cov_to_add = []
for i in range(len(ctylist)):
    seri = policy3.loc[coverage_df.index[i]]
    for fips in ctylist[i]:
        seri['Coverage'] = fips
        cov_to_add.append(seri.copy())

In [27]:
policy3_new = policy3.replace({'Coverage':{'State-wide':np.nan}})
policy3_new = policy3_new[policy3_new['Coverage'].isna()]
policy3_new.fillna('0', inplace=True)
policy3_new = policy3_new.append(pd.DataFrame(cov_to_add), ignore_index=True)
policy3_new['fips'] = policy3_new['fips']
policy3_new.head()

,State,Effective Date,Coverage,State of Emergency Declaration,Shelter-in-place Order,business
0,41,2020-02-28,0,False,False,False
1,53,2020-02-29,0,True,False,False
2,12,2020-03-01,0,True,False,False
3,01,2020-03-02,0,False,False,False
4,06,2020-03-04,0,True,False,False


In [28]:
policy3_new['Coverage'].unique()

array(['0', '53033', '53061', '53053', '05125', '05069', '05119', '05053',
       '06041', '06067', '06077', '06079', '06085', '06095', '06061',
       '06013', '06087', '06073', '36027', '36071', '36043', '36111',
       '47037', '36119', '36103', '36059', '06001', '06075', '06081',
       '16013', '20085', '20091', '08113', '15003', '15009', '31025',
       '31055', '31153', '04013', '04021', '04017', '04005', '04009',
       '15007', '02020', '13095', '20209', '20103', '29047', '29165',
       '29095', '53077', '08031', '08013', '08097', '29189', '42101',
       '42003', '42017', '42029', '42045', '42089', '42091', '48113',
       '48027', '48061', '48231', '48429', '48281', '48121', '48257',
       '48041', '48167', '48141', '48453', '48491', '48015', '48069',
       '48351', '48085', '48201', '48215', '48397', '55079', '12001',
       '13045', '13185', '13227', '20045', '29183', '29099', '29037',
       '47093', '47157', '08059', '08001', '08005', '08069', '08123',
       '08035',

In [124]:
fips_to_st = {}
for dic in po_st:
    fips_to_st[dic['fips']] = dic['state']

for city in excep:
    print(city[1]+',',fips_to_st[city[0]])

Scarsdale, New York
Osceola, Wisconsin
the Oakland, California
Antioch, California
Los Angeles Unified, California
Chaffey Unified, California
Etiwanda, California
Fontana Unified, California
Ontario-Montclair, California
Alta Loma Unified, California
Los Alamitos Unified, California
Washington Unified School Distrcts., California
Wellesley, Massachusetts
Framingham, Massachusetts
Boston, Massachusetts
Warwick, New York
St Paul, Minnesota
Hoboken, New Jersey
New York City, New York
Burlington, Vermont
Tucson, Arizona
Phoenix, Arizona
Santa Clara., California
Kansas City, Kansas
New York City, New York
Athens-Clarke, Georgia
Chattanooga, Tennessee
New Orleans, Louisiana
Knoxville, Tennessee
Columbus, Mississippi
Everett, Washington
Tupelo, Mississippi
Oxford, Mississippi
Belton, Missouri
Kansas City, Missouri
Nashville, Tennessee
Edmonds, Washington
Brunswick, Maine
Waco, Texas
Mclennan, Texas
Forney, Texas
Verde, Texas
Bryan, Texas
College, Texas
Station, Texas
San Antonio, Texas
Fort 

In [102]:
cov_dict = dict(zip(coverage_list, cov_parsed))
newcol = []
for cov in list(policy3['Coverage']):
    newcol.append(cov_dict[cov])

In [10]:
po_to_fips = {}
for dic in po_st:
    po_to_fips[dic['postalCode']] = dic['fips']

fips_key = pd.read_csv(f'{homedir}/data/us/processing_data/fips_key.csv', encoding='latin-1')
fips_key['FIPS'] = fips_key['FIPS'].apply(correct_FIPS)
fips_key.replace({'ST':po_to_fips}, inplace=True)
fips_key.head()

,FIPS,MSA/PMSA NECMA,ST,COUNTY
0,01001,5240,01,Autauga County
1,01003,5160,01,Baldwin County
2,01005,0,01,Barbour County
3,01007,0,01,Bibb County
4,01009,1000,01,Blount County


## Create curated DataFrame

In [8]:
policy = pd.DataFrame(columns = ['fips', 'emergency', 'safeathome', 'business'])
policy['fips'] = demo['fips']
policy.head()

,fips,emergency,safeathome,business,travel
0,01001,NaN,NaN,NaN,NaN
1,01003,NaN,NaN,NaN,NaN
2,01005,NaN,NaN,NaN,NaN
3,01007,NaN,NaN,NaN,NaN
4,01009,NaN,NaN,NaN,NaN
